# Creating neural network from scratch with Python

Reference link for the code snippet- https://heartbeat.fritz.ai/building-a-neural-network-from-scratch-using-python-part-1-6d399df8d432

Reference link for the dataset - https://archive.ics.uci.edu/ml/datasets/Statlog+%28Heart%29

## Importing the dataset 

In [2]:
#Importing Pandas module
import csv
import pandas as pd

#Add header names
headers =  ['age', 'sex','chest_pain','resting_blood_pressure',  
        'serum_cholestoral', 'fasting_blood_sugar', 'resting_ecg_results',
        'max_heart_rate_achieved', 'exercise_induced_angina', 'oldpeak',"slope of the peak",
        'num_of_major_vessels','thal', 'heart_disease']

#Import dataset in the form of a dataframe
heart_df = pd.read_csv('https://github.com/prateekef/workshopdata/blob/main/heart.dat', sep=' ', names=headers)

#Looking at the first five rows of the dataset
heart_df.head()

,age,sex,chest_pain,resting_blood_pressure,serum_cholestoral,fasting_blood_sugar,resting_ecg_results,max_heart_rate_achieved,exercise_induced_angina,oldpeak,slope of the peak,num_of_major_vessels,thal,heart_disease
0,<!DOCTYPE,html>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,<html,"lang=""en""","data-color-mode=""auto""","data-light-theme=""light""","data-dark-theme=""dark"">",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,<head>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,<meta,"charset=""utf-8"">",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,<link,"rel=""dns-prefetch""","href=""https://github.githubassets.com"">",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Getting shape of the data
heart_df.shape

In [ ]:
#Looking for the null values through the dataset because neural net require non null variables
heart_df.isna().sum()

## Splitting the dataset into training and test parts

In [ ]:
#Importing essential packages
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

#Splitting data into independant and depedant variables

X = heart_df.drop(columns=['heart_disease']) #Independant data variables

#replace target class with 0 and 1 
#1 means "have heart disease" and 0 means "do not have heart disease"
heart_df['heart_disease'] = heart_df['heart_disease'].replace(1, 0)
heart_df['heart_disease'] = heart_df['heart_disease'].replace(2, 1)

y_label = heart_df['heart_disease'].values.reshape(X.shape[0], 1) #Dependant or target variable

#Split data into train and test set
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y_label, test_size=0.2, random_state=2)

#Standardize the dataset
sc = StandardScaler()
sc.fit(Xtrain)
Xtrain = sc.transform(Xtrain)
Xtest = sc.transform(Xtest)

print(f"Shape of train set is {Xtrain.shape}")
print(f"Shape of test set is {Xtest.shape}")
print(f"Shape of train label is {ytrain.shape}")
print(f"Shape of test labels is {ytest.shape}")

## Implementing Neural network class

In [ ]:

class NeuralNet():
    '''
    A two layer neural network
    '''
        
    def __init__(self, layers=[13,8,1], learning_rate=0.001, iterations=100):
        self.params = {}
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.loss = []
        self.sample_size = None
        self.layers = layers
        self.X = None
        self.y = None
                
    def init_weights(self):
        '''
        Initialize the weights from a random normal distribution
        '''
        np.random.seed(1) # Seed the random number generator
        self.params["W1"] = np.random.randn(self.layers[0], self.layers[1]) 
        self.params['b1']  =np.random.randn(self.layers[1],)
        self.params['W2'] = np.random.randn(self.layers[1],self.layers[2]) 
        self.params['b2'] = np.random.randn(self.layers[2],)
        
    def relu(self,Z):
        '''
        The ReLu activation function is to performs a threshold
        operation to each input element where values less 
        than zero are set to zero.
        '''
        return np.maximum(0,Z)
    
    def sigmoid(self,Z):
        '''
        The sigmoid function takes in real numbers in any range and 
        squashes it to a real-valued output between 0 and 1.
        '''
        return 1.0/(1.0+np.exp(-Z))
    
    def entropy_loss(self,y, yhat):
        nsample = len(y)
        loss = -1/nsample * (np.sum(np.multiply(np.log(yhat), y) + np.multiply((1 - y), np.log(1 - yhat))))
        return loss
    
    def forward_propagation(self):
        '''
        Performs the forward propagation
        '''
        
        Z1 = self.X.dot(self.params['W1']) + self.params['b1']
        A1 = self.relu(Z1)
        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        yhat = self.sigmoid(Z2)
        loss = self.entropy_loss(self.y,yhat)

        # save calculated parameters     
        self.params['Z1'] = Z1
        self.params['Z2'] = Z2
        self.params['A1'] = A1

        return yhat,loss
    
    def back_propagation(self,yhat):
        '''
        Computes the derivatives and update weights and bias according.
        '''
        def dRelu(x):
            x[x<=0] = 0
            x[x>0] = 1
            return x
        
        dl_wrt_yhat = -(np.divide(self.y,yhat) - np.divide((1 - self.y),(1-yhat)))
        dl_wrt_sig = yhat * (1-yhat)
        dl_wrt_z2 = dl_wrt_yhat * dl_wrt_sig

        dl_wrt_A1 = dl_wrt_z2.dot(self.params['W2'].T)
        dl_wrt_w2 = self.params['A1'].T.dot(dl_wrt_z2)
        dl_wrt_b2 = np.sum(dl_wrt_z2, axis=0)

        dl_wrt_z1 = dl_wrt_A1 * dRelu(self.params['Z1'])
        dl_wrt_w1 = self.X.T.dot(dl_wrt_z1)
        dl_wrt_b1 = np.sum(dl_wrt_z1, axis=0)
        
        #update the weights and bias
        self.params['W1'] = self.params['W1'] - self.learning_rate * dl_wrt_w1
        self.params['W2'] = self.params['W2'] - self.learning_rate * dl_wrt_w2
        self.params['b1'] = self.params['b1'] - self.learning_rate * dl_wrt_b1
        self.params['b2'] = self.params['b2'] - self.learning_rate * dl_wrt_b2
        
    
    def fit(self, X, y):
        '''
        Trains the neural network using the specified data and labels
        '''
        self.X = X
        self.y = y
        self.init_weights() #initialize weights and bias


        for i in range(self.iterations):
            yhat, loss = self.forward_propagation()
            self.back_propagation(yhat)
            self.loss.append(loss)
            
    def predict(self, X):
        '''
        Predicts on a test data
        '''
        Z1 = X.dot(self.params['W1']) + self.params['b1']
        A1 = self.relu(Z1)
        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        pred = self.sigmoid(Z2)
        return np.round(pred) 
    
    def acc(self, y, yhat):
        '''
        Calculates the accuracy between the predicted value and the truth labels
        '''
        acc = int(sum(y == yhat) / len(y) * 100)
        return acc


    def plot_loss(self):
        '''
        Plots the loss curve
        '''
        plt.plot(self.loss)
        plt.xlabel("Iteration")
        plt.ylabel("logloss")
        plt.title("Loss curve for training")
        plt.show()

In [ ]:
#Create object of class Neural net
nn = NeuralNet(layers=[13,8,1], learning_rate=0.001, iterations=100) # create the NN model

#Fit the train dataset to this object - training the model
nn.fit(Xtrain, ytrain) #train the model

In [ ]:
#Plot the loss 
nn.plot_loss()

In [ ]:
#Predict the train and test data 
train_pred = nn.predict(Xtrain)
test_pred = nn.predict(Xtest)

#Calculate the accuracy of the train and test datasets
print("Train accuracy is {}".format(nn.acc(ytrain, train_pred)))
print("Test accuracy is {}".format(nn.acc(ytest, test_pred)))

In [ ]:
nn.loss[len(nn.loss)-1]

## Creating neural net with Python libraries

## With help of Scikit-learn

In [ ]:
#With scikit learn - with multilayer perceptron classifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

sknet = MLPClassifier(hidden_layer_sizes=(8), learning_rate_init=0.001, max_iter=100)

In [ ]:
#Fit the data to the classifier model
sknet.fit(Xtrain, ytrain)
preds_train = sknet.predict(Xtrain)
preds_test = sknet.predict(Xtest)

#Print the accuracy of the train and test datasets
print("Train accuracy of sklearn neural network: {}".format(round(accuracy_score(preds_train, ytrain),2)*100))
print("Test accuracy of sklearn neural network: {}".format(round(accuracy_score(preds_test, ytest),2)*100))

Reference for how to save and load scikit learn model with pickle - https://www.geeksforgeeks.org/saving-a-machine-learning-model/#:~:text=There%20are%20two%20ways%20we,serializing%20a%20Python%20object%20structure.&text=pickle.,you%20simply%20use%20dump().

In [ ]:
! conda install tensorflow

In [ ]:
! pip install keras

## With the help of Keras

In [ ]:
#With Keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

#Define the model
model = Sequential()
model.add(Dense(8,input_shape=(13,)))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the model
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Fitting the model to data - Training 
model.fit(Xtrain, ytrain, epochs=100, verbose=1)

#Calculate the train accuracy
train_acc = model.evaluate(Xtrain, ytrain, verbose=1)[1]
# print(train_acc)

#Calculate the test accuracy
test_acc = model.evaluate(Xtest, ytest, verbose=1)[1]
# print(test_acc)

print("Train accuracy of keras neural network: {}".format(round((train_acc * 100), 2)))
print("Test accuracy of keras neural network: {}".format(round((test_acc * 100),2)))

## How to save Keras model

In [ ]:
#How to save Keras models
#save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

## How to load the Keras model

In [ ]:
#How to load saved keras model and use it again
from numpy import loadtxt
from tensorflow.keras.models import load_model
 
#Load model
model = load_model("model.h5")

#Summarize model
model.summary()

Reference link for how to save models and load them again - https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [ ]:
np.round(model.predict(Xtest),0)